## Chat

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
model = ChatGroq(model="llama-3.3-70b-versatile")

In [ ]:
messages = [
    (
        "system", "あなたは役立つAIアシスタントです。"
    ),
    (
        "human", "あなたについて教えてください。"
    )
]

In [ ]:
res = model.invoke(messages)
print(res.content)

## Chat with memory

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [ ]:
model = ChatGroq(model_name="llama-3.3-70b-versatile")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは親切なAIアシスタントです。"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

chain = prompt | model

store = {}
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

config = {"configurable": {"session_id": "user001"}}

In [ ]:
res = with_message_history.invoke(
    {"input": "こんにちは！私の名前は太郎です。"},
    config=config
)
print(res.content)

In [ ]:
res = with_message_history.invoke(
    {"input": "私の名前を覚えていますか？"},
    config=config
)
print(res.content)

In [ ]:
# 履歴の確認
history = get_session_history("user001")
for message in history.messages:
    print(f"{message.type}: {message.content}")

## RAG

In [ ]:
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
docs = [
    """Groqとは
Groq は、超低レイテンシかつ高スループットの AI 推論専用プロセッサ（LPU: Language Processing Unit） を開発している米国の企業である。特に大規模言語モデル (LLM) や生成 AI の推論に特化しており、従来の GPU や CPU とは異なるアーキテクチャを採用している。""",
    """特徴
1. アーキテクチャ設計
  - 単一命令ストリーム：Groq のチップは GPU のように多数のスレッドを並列に実行するのではなく、単一の制御フローで大量の演算を同期的に処理する方式を採用している。
  - 行列演算に最適化：ディープラーニングのコアとなる行列・ベクトル演算を最適化したハードウェア設計である。
  - 固定パイプライン：メモリ帯域やキャッシュ制御のオーバーヘッドを排除し、決まったパターンの計算を極めて効率よく処理する。
2. 性能
  - 超低レイテンシ：数ミリ秒以下での推論応答を実現可能であり、GPU ベースの処理と比べて遅延が大幅に小さい。
  - 高スループット：並列推論リクエストを処理する能力に優れており、特にリアルタイム応答が必要な LLM サービスに適している。
  - 消費電力あたりの効率：GPU に比べて推論性能あたりのエネルギー効率が高いとされる。
3. ソフトウェアスタック
  - GroqWare：専用のコンパイラやランタイム環境を提供し、TensorFlow / PyTorch などの既存 ML フレームワークからモデルを Groq LPU に移植可能にしている。
  - 推論最適化：モデルを LPU 向けに変換し、最大限の効率を引き出す。""",
    """利用分野
  - 大規模言語モデル (LLM) 推論：ChatGPT のような会話型 AI の高速応答
  - 金融分野：リアルタイムのリスク評価やアルゴ取引
  - 自動運転・ロボティクス：ミリ秒単位での意思決定が必要な制御システム
  - クラウド AI サービス：低レイテンシな API 提供""",
    """GPU / TPU との比較
  - GPU：汎用的だが、スレッド制御やキャッシュ管理のオーバーヘッドがある。
  - TPU：Google 専用でクラウド特化。行列演算に強いが、柔軟性に欠ける。
  - Groq LPU：推論専用。構造を単純化することで遅延を極限まで削減している。"""
]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.create_documents(docs)
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
vector = Chroma.from_documents(documents, embeddings)
retriever = vector.as_retriever()

In [ ]:
rag_prompt = ChatPromptTemplate.from_template("""
以下のコンテキスト情報のみを使用して、質問に答えてください。

<context>
{context}
</context>

Question: {input}
""")
model = ChatGroq(model="llama-3.3-70b-versatile")
document_chain = create_stuff_documents_chain(model, rag_prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
res = retrieval_chain.invoke({"input": "Groqとは何ですか？"})
print(res["answer"])

In [ ]:
res["context"]

### RAG + memory

In [ ]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは厳格な教授です。質問に対して厳しめに回答します。"),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    model, retriever, contextualize_q_prompt
)

In [ ]:
qa_system_prompt = """
以下のコンテキスト情報のみを使用して、質問に答えてください。
コンテキスト情報から答えがわからない場合は、正直にそう言ってください。

<context>
{context}
</context>
"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)
rag_with_history_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
conversational_rag_chain = RunnableWithMessageHistory(
    rag_with_history_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)
config_rag = {"configurable": {"session_id": "rag_user002"}}

In [ ]:
conversational_rag_chain

In [ ]:
res = conversational_rag_chain.invoke(
    {"input": "こんにちは、私は新入生の圭一です。Groqとは何ですか？"},
    config=config_rag
)
print(res["answer"])

In [ ]:
res = conversational_rag_chain.invoke(
    {"input": "そのソフトウェアスタックについて教えてください。"},
    config=config_rag
)
print(res["answer"])

In [ ]:
res = conversational_rag_chain.invoke(
    {"input": "どのような分野で活躍しますか？"},
    config=config_rag
)
print(res["answer"])

In [ ]:
res = conversational_rag_chain.invoke(
    {"input": "ところで、私の名前を覚えていますか？"},
    config=config_rag
)
print(res["answer"])

In [ ]:
res = conversational_rag_chain.invoke(
    {"input": "あなたは？"},
    config=config_rag
)
print(res["answer"])

In [ ]:
res = conversational_rag_chain.invoke(
    {"input": "あれ？教授では？"},
    config=config_rag
)
print(res["answer"])